<a href="https://colab.research.google.com/github/sandeep1847e/MSc-Project/blob/main/codes/Jet_2_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install uproot
# !pip install fastjet
# !pip install awkward

In [7]:
# refrences
# https://scikit-learn.org/1.5/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# https://awkward-array.org/doc/main/
# 

In [1]:
import uproot
import fastjet as fj
import numpy as np
import awkward as ak

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
import requests

# URL of the .root file
url = "https://github.com/sandeep1847e/MSc-Project/blob/main/Events/pp24tops100E.root?raw=true"

# Download the file
response = requests.get(url)
with open("pp24tops100E.root", "wb") as file:
    file.write(response.content)

print("Download complete!")


In [ ]:
file = uproot.open("pp24tops100E.root")
print(file.keys())
tree = file["Delphes;1"]
print(tree.keys())


In [5]:

# Open the ROOT file using uproot
# # file = uproot.open("pp24tops100E.root")
# file = uproot.open("/home/sandeep/Desktop/project/root/pp24tops100E.root")

# tree = file["Delphes;1"]
# tree.keys()

In [6]:
# # Access the Jet branch
# jets = tree["Jet"].arrays()
# print(jets)
# len(jets)


In [4]:
jet=tree['Jet'].arrays()
# len(jet)

In [5]:
jet_pt=jet['Jet.PT']
jet_eta=jet['Jet.Eta']
jet_phi=jet['Jet.Phi']
jet_mass=jet['Jet.Mass']
# jet_pt
# len(jet_pt)

In [ ]:
print(len(jet_pt[0]))
print(len(jet_eta[0]))
print(len(jet_phi[0]))


In [10]:
# j=0
# for i in range(len(jet_pt)):
#   j+=1
#   print(i)
#   print(jet_pt[i])

In [11]:
# for i, jets in enumerate(jet_pt):
#     print(f"Event {i+1} Jets (pT):")
#     for j, pt in enumerate(jets):
#         print(f"  Jet {j+1}: {pt} GeV")
#     print("-" * 30)




In [ ]:
import pandas as pd
dataframes = []

# Looping through each event
for event_no, (jets_pt, jets_eta, jets_phi,jet_mass) in enumerate(zip(jet_pt, jet_eta, jet_phi,jet_mass), start=1):
    # Create a DataFrame for each event
    event_df = pd.DataFrame({
        'event_no': [event_no] * len(jets_pt),  # Event number for each jet
        'jet_no': list(range(1, len(jets_pt) + 1)),  # Jet number
        'jet_pt': jets_pt,  # Jet transverse momentum
        'jet_eta': jets_eta,  # Jet pseudorapidity
        'jet_phi': jets_phi,   # Jet azimuthal angle
        'jet_mass':jet_mass   #Jet mass
    })
    # print(event_df)
    # Append the event DataFrame to the list
    dataframes.append(event_df)

# Concatenating all DataFrames into a single DataFrame
final_df = pd.concat(dataframes, ignore_index=True)
print(final_df)

In [ ]:
len(final_df)

In [14]:
# distance_threshold=300
# all_graphs = []

# for event_no in final_df['event_no'].unique():
#     event_data = final_df[final_df['event_no'] == event_no]
#     G_event = nx.Graph()

#     # Add nodes for each jet in the event
#     for idx, row in event_data.iterrows():
#         G_event.add_node(row['jet_no'], pt=row['jet_pt'], eta=row['jet_eta'], phi=row['jet_phi'])

#     # Add edges based on distance
#     for i in range(len(event_data)):
#         for j in range(i + 1, len(event_data)):
#             # Calculate distance (using pt, eta, phi)
#             pt1, eta1, phi1 = event_data.iloc[i][['jet_pt', 'jet_eta', 'jet_phi']]
#             pt2, eta2, phi2 = event_data.iloc[j][['jet_pt', 'jet_eta', 'jet_phi']]
#             distance = ((eta1 - eta2)**2 + (phi1 - phi2)**2 + (pt1 - pt2)**2)**0.5  # Example metric

#             if distance < distance_threshold:
#                 G_event.add_edge(event_data.iloc[i]['jet_no'], event_data.iloc[j]['jet_no'])

#     all_graphs.append(G_event)

# # Now `all_graphs` contains separate graphs for each event


In [15]:
# for i, G_event in enumerate(all_graphs[1:31]):
#     plt.figure(figsize=(4, 4))
#     pos = nx.spring_layout(G_event)
#     nx.draw(G_event, pos, with_labels=True, node_size=50, node_color='blue', edge_color='gray', alpha=0.5)
#     plt.title(f'Jet Connectivity Graph for Event {i+1}')
#     plt.show()


In [20]:
jet_pt=jet['Jet.PT']
jet_eta=jet['Jet.Eta']
jet_phi=jet['Jet.Phi']
jet_mass=jet['Jet.Mass']



In [21]:

from sklearn.preprocessing import MinMaxScaler

dataframes = []

# Looping through each event
for event_no, (jets_pt, jets_eta, jets_phi, jet_mass) in enumerate(zip(jet_pt, jet_eta, jet_phi, jet_mass), start=1):
    # Create a DataFrame for each event
    event_df = pd.DataFrame({
        'event_no': [event_no] * len(jets_pt),  # Event number for each jet
        'jet_no': list(range(1, len(jets_pt) + 1)),  # Jet number
        'jet_pt': jets_pt,  # Jet transverse momentum
        'jet_eta': jets_eta,  # Jet pseudorapidity
        'jet_phi': jets_phi,  # Jet azimuthal angle
        'jet_mass': jet_mass  # Jet mass
    })
    dataframes.append(event_df)

# Concatenating all DataFrames into a single DataFrame
final_df = pd.concat(dataframes, ignore_index=True)

# Normalize jet properties
scaler = MinMaxScaler()
final_df[['jet_pt', 'jet_eta', 'jet_phi','jet_mass']] = scaler.fit_transform(final_df[['jet_pt', 'jet_eta', 'jet_phi','jet_mass']])

distance_threshold = 0.5  
all_graphs = []

for event_no in final_df['event_no'].unique():
    event_data = final_df[final_df['event_no'] == event_no]
    G_event = nx.Graph()

    # Add nodes for each jet in the event
    for idx, row in event_data.iterrows():
        G_event.add_node(row['jet_no'], pt=row['jet_pt'], eta=row['jet_eta'], phi=row['jet_phi'])

    # Add edges based on distance
    for i in range(len(event_data)):
        for j in range(i + 1, len(event_data)):
            # Calculate distance (using normalized pt, eta, phi)
            pt1, eta1, phi1 = event_data.iloc[i][['jet_pt', 'jet_eta', 'jet_phi']]
            pt2, eta2, phi2 = event_data.iloc[j][['jet_pt', 'jet_eta', 'jet_phi']]
            distance = ((eta1 - eta2)**2 + (phi1 - phi2)**2 + (pt1 - pt2)**2)**0.5

            if distance < distance_threshold:
                G_event.add_edge(event_data.iloc[i]['jet_no'], event_data.iloc[j]['jet_no'])

    all_graphs.append(G_event)


In [ ]:
# Now `all_graphs` contains separate graphs for each event
for i, G_event in enumerate(all_graphs[0:6]):
    plt.figure(figsize=(6,4))
    pos = nx.spring_layout(G_event)
    node_sizes = [G_event.degree(n) * 100 for n in G_event.nodes()]  # Adjust node size by degree
    node_colors = [G_event.nodes[n]['pt'] for n in G_event.nodes()]  # Color nodes by pt

    # Draw edges
    nx.draw_networkx_edges(G_event, pos, alpha=0.5, edge_color='gray')

    # Draw labels
    nx.draw_networkx_labels(G_event, pos)
    plt.title(f'Jet Connectivity Graph for Event {i+1}')

plt.show()

In [25]:
for G_event in all_graphs:
    # Create and attach the adjacency matrix as an attribute to the graph
    G_event.graph['adjacency_matrix'] = nx.to_numpy_array(G_event, nodelist=sorted(G_event.nodes()), weight="weight")


In [27]:
for G_event in all_graphs[0:15]:
    adjacency_matrix = G_event.graph['adjacency_matrix']
    print(adjacency_matrix)


[[0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1.]
 [1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1.]
 [0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0.]
 [1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0.]
 [0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0.]
 [1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1.]
 [0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0.]
 [1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0.]
 [0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1.]
 [1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.]]
[[0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 1. 0. 1. 1.]
 [

In [28]:
import numpy as np

# Step 1: Determine the maximum number of jets in any event
max_jets = max(len(G_event.nodes) for G_event in all_graphs)

# Step 2: Pad each adjacency matrix to match the max_jets dimension
padded_adjacency_matrices = []

for G_event in all_graphs:
    # Get the adjacency matrix for the current graph
    adjacency_matrix = nx.to_numpy_array(G_event, nodelist=sorted(G_event.nodes()), weight="weight")
    
    # Get the current number of jets for this event
    num_jets = adjacency_matrix.shape[0]
    
    # If the matrix is smaller than max_jets, pad it
    if num_jets < max_jets:
        padded_matrix = np.zeros((max_jets, max_jets))
        padded_matrix[:num_jets, :num_jets] = adjacency_matrix
    else:
        padded_matrix = adjacency_matrix
    
    padded_adjacency_matrices.append(padded_matrix)

# Now `padded_adjacency_matrices` contains square matrices of shape (max_jets, max_jets) for each event


In [29]:
padded_adjacency_matrices[0:5]

[array([[0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.],
        [1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0.],
        [0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0.],
        [0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.],
        [1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0.],
        [1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0.],
        